In [194]:
import sys
print(sys.version)

3.5.2 (v3.5.2:4def2a2901a5, Jun 26 2016, 10:47:25) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]


In [2]:
# посмотрим информацию об операционной системе, на которой мы работаем.
import platform
platform.uname()

uname_result(system='Darwin', node='MacAir.local', release='15.5.0', version='Darwin Kernel Version 15.5.0: Tue Apr 19 18:36:36 PDT 2016; root:xnu-3248.50.21~8/RELEASE_X86_64', machine='x86_64', processor='i386')

## Работа с файлами

In [75]:
f = open("/tmp/example.txt", "w")
f.write("Технопарк\n")
f.close()

In [4]:
# открываем дескриптор файла для чтения
f = open("/tmp/example.txt", "r")

# читаем содержимое полностью.
data = f.read()

# обязательно закрываем!
f.close()

print(data)

Технопарк
МГТУ



In [77]:
# используя context-manager
with open("/tmp/example.txt", "a") as f:
    f.write("МГТУ\n")

In [79]:
with open("/tmp/example.txt", "r") as f:
    print(f.readlines())

['Технопарк\n', 'МГТУ\n']


In [82]:
# читаем файл по строке, не загружая его полность в память
with open("/tmp/example.txt") as f:
    for line in f:
        print(repr(line))

'Технопарк\n'
'МГТУ\n'


## Работа с директориями

In [215]:
import os

os.mkdir("/tmp/park-python")

In [216]:
try:
    os.rmdir("/tmp/park-python")
except IOError as err:
    print(err)

In [217]:
path = "/tmp/park-python/lectures/04"
if not os.path.exists(path):
    os.makedirs(path)

In [218]:
os.rmdir("/tmp/park-python")

OSError: [Errno 66] Directory not empty: '/tmp/park-python'

In [219]:
import shutil
shutil.rmtree("/tmp/park-python")

## stdin, stdout, stderr

In [197]:
import sys
print(sys.stdin)
print(sys.stdout)
print(sys.stderr)

<_io.TextIOWrapper name='<stdin>' mode='r' encoding='UTF-8'>


In [2]:
print(sys.stdin.fileno())

0


In [3]:
print(sys.stdout.fileno())

UnsupportedOperation: IOStream has no fileno.

Так как дескрипторы stdout и stderr переопределены в Jupyter notebook. Давайте посмотрим куда они ведут:

In [193]:
sys.stdout.write("where am I")

NameError: name 'sys' is not defined

А ведут они как раз в этот ноутбук:)

# Отладка

In [5]:
def find_sum(a, b):
    import pdb
    pdb.set_trace()    
    return a + b

find_sum(1, 4)

> <ipython-input-5-22c1ca74b9d8>(4)find_sum()
-> return a + b
(Pdb) ll
  1  	def find_sum(a, b):
  2  	    import pdb
  3  	    pdb.set_trace()
  4  ->	    return a + b
(Pdb) locals()
{'a': 1, 'b': 4, 'pdb': <module 'pdb' from '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/pdb.py'>}
(Pdb) c


5

Гораздо удобнее использовать `ipdb`, иногда достаточно просто ф-ии `print()`

Полезные библиотеки для отладки (debugging) - https://github.com/vinta/awesome-python#debugging-tools

# Тестирование

**Зачем?**

In [21]:
def get_max_length_word(sentence):
    longest_word = None
    words = sentence.split()
    for word in words:
        if not longest_word or len(word) > len(longest_word):
            longest_word = word
    return longest_word

Что может пойти не так? Да все что угодно:

* SyntaxError
* Ошибка в логике
* Обратная несовместимость новой версии используемой библиотеки
* ...

Через полгода после запуска приложения без тестов изменения в код большого приложения вносить очень страшно!

Некоторые даже используют TDD - Test-Driven Development.

## unittest

In [23]:
import unittest

class LongestWordTestCase(unittest.TestCase):

    def test_sentences(self):
        sentences = [
            ["Beautiful is better than ugly.", "Beautiful"],
            ["Complex is better than complicated.", "complicated"]
        ]
        for sentence, correct_word in sentences:
            self.assertEqual(get_max_length_word(sentence), correct_word)

# Обычно в реальных проектах использует механизм автоматического нахождения тестов (discover).
suite = unittest.defaultTestLoader.loadTestsFromTestCase(LongestWordTestCase)
unittest.TextTestRunner().run(suite)

F
FAIL: test_sentences (__main__.LongestWordTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-23-85c6de7675a8>", line 11, in test_sentences
    self.assertEqual(get_max_length_word(sentence), correct_word)
AssertionError: 'complicated.' != 'complicated'
- complicated.
?            -
+ complicated


----------------------------------------------------------------------
Ran 1 test in 0.024s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

<table border="1" class="docutils" align="left">
<colgroup>
<col width="48%">
<col width="34%">
<col width="18%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Method</th>
<th class="head">Checks that</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>assertEqual(a, b)</td>
<td><code class="docutils literal"><span class="pre">a</span> <span class="pre">==</span> <span class="pre">b</span></code></td>
</tr>
<tr class="row-odd"><td>assertNotEqual(a, b)</td>
<td><code class="docutils literal"><span class="pre">a</span> <span class="pre">!=</span> <span class="pre">b</span></code></td>
</tr>
<tr class="row-even"><td>assertTrue(x)</td>
<td><code class="docutils literal"><span class="pre">bool(x)</span> <span class="pre">is</span> <span class="pre">True</span></code></td>
</tr>
<tr class="row-odd"><td>assertFalse(x)</td>
<td><code class="docutils literal"><span class="pre">bool(x)</span> <span class="pre">is</span> <span class="pre">False</span></code></td>
</tr>
<tr class="row-even"><td>assertIsNone(x)</td>
<td><code class="docutils literal"><span class="pre">x</span> <span class="pre">is</span> <span class="pre">None</span></code></td>
</tr>
<tr class="row-odd"><td>assertIsNotNone(x)</td>
<td><code class="docutils literal"><span class="pre">x</span> <span class="pre">is</span> <span class="pre">not</span> <span class="pre">None</span></code></td>
</tr>
<tr class="row-even"><td>assertIsInstance(a, b)</td>
<td><code class="docutils literal"><span class="pre">isinstance(a,</span> <span class="pre">b)</span></code></td>
</tr>
<tr class="row-even"><td colspan="2">И другие... https://docs.python.org/3.5/library/unittest.html</td>
</tr>
</tbody>
</table>

## unittest.mock

Как тестировать код, выполняющий внешние вызовы: чтение файла, запрос содержимого URL?

In [17]:
import requests

def get_location_city(ip):
    data = requests.get("https://freegeoip.net/json/{ip}".format(ip=ip)).json()
    return data["city"]

def get_ip():
    data = requests.get("https://httpbin.org/ip").json()
    return data["origin"]

get_location_city(get_ip())

'Moscow'

Для начала посмотрим что такое monkey patching.

In [18]:
import math

def fake_sqrt(num):
    return num

original_sqrt = math.sqrt
math.sqrt = fake_sqrt

print(math.sqrt(16))

math.sqrt = original_sqrt

16


In [19]:
math.sqrt(16)

4.0

In [20]:
import unittest
from unittest.mock import patch, Mock


class FakeIPResponse:

    def json(self):
        return {"origin": "127.0.0.1"}


class LongestWordTestCase(unittest.TestCase):

    @patch('requests.get', Mock(return_value=FakeIPResponse()))
    def test_get_ip(self):
        self.assertEqual(get_ip(), "127.0.0.1")

suite = unittest.defaultTestLoader.loadTestsFromTestCase(LongestWordTestCase)
unittest.TextTestRunner().run(suite)


.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Библиотека `coverage` позволяет оценить степень покрытия кода тестами.

Помимо unit-тестирования существует масса других типов тестов:
* Интеграционные
* Функциональные
* Тестирование производительности (бенчмарки)
* ...

Полезные библиотеки для тестирования - https://github.com/vinta/awesome-python#testing

# Многопоточность

Что такое процесс?

UNIX является многозадачной операционной системой. Это означает, что одновременно может быть запущена более чем одна программа. Каждая программа, работающая в некоторый момент времени, называется процессом.

http://www.kharchuk.ru/%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D0%B8/15-unix-foundations/80-unix-processes

In [60]:
STEPS = 10000000

# Простая программа, складывающая числа.
def worker(steps):
    count = 0
    for i in range(steps):
        count += 1
    return count

print(worker(STEPS))
    
%timeit -n1 worker(STEPS)

print("Напомните преподавателю показать top")

10000000
1 loop, best of 3: 1.37 s per loop
Напомните преподавателю показать top


<div style="float:left;margin:0 10px 10px 0" markdown="1">![title](img/g3.png)</div>

Что такое поток?

Многопоточность является естественным продолжением многозадачности. Каждый из процессов может выполнятся в несколько потоков. Программа выше исполнялась в одном процессе в главном потоке.

http://www.cs.miami.edu/home/visser/Courses/CSC322-09S/Content/UNIXProgramming/UNIXThreads.shtml

Логичный шаг предположить, что 2 потока выполнят программу выше быстрее. Проверим?

In [63]:
import threading
import queue

result_queue = queue.Queue()

STEPS = 10000000
NUM_THREADS = 2

def worker(steps):
    count = 0
    for i in range(steps):
        count += 1
    result_queue.put(count)


def get_count_threaded():    
    count = 0
    threads = []

    for i in range(NUM_THREADS):
        t = threading.Thread(target=worker, args=(STEPS//NUM_THREADS,))
        threads.append(t)
        t.start()

    for i in range(NUM_THREADS):
        count += result_queue.get()

    return count

print(get_count_threaded())
%timeit -n1 get_count_threaded()

10000000
1 loop, best of 3: 1.43 s per loop


<div style="float:left;margin:0 10px 10px 0" markdown="1">![title](img/g4.png)</div>

GIL

https://jeffknupp.com/blog/2012/03/31/pythons-hardest-problem/

Ок. Неужели выхода нет? Есть - multiprocessing

## Мультипроцессинг

In [65]:
import multiprocessing

NUM_PROCESSES = 2
STEPS = 10000000

result_queue = multiprocessing.Queue()

def worker(steps):
    count = 0
    for i in range(steps):
        count += 1

    result_queue.put(count)


def get_count_in_processes():    
    count = 0
    processes = []
    for i in range(NUM_PROCESSES):
        p = multiprocessing.Process(target=worker, args=(STEPS//NUM_PROCESSES,))
        processes.append(t)
        p.start()

    for i in range(NUM_THREADS):
        count += result_queue.get()

    return count

print(get_count_in_processes())
%timeit -n1 get_count_in_processes()

10000000
1 loop, best of 3: 736 ms per loop


**Зачем тогда нужны потоки?**

Все потому что не все задачи CPU-bound. Есть IO-bound задачи, которые прекрасно параллелятся на несколько CPU. Кто приведет пример?

**TODO: примеры**

Чтобы не быть голословными, поднимем HTTP-сервер на порту 8000. По адресу http://localhost:8000 будет отдаваться небольшой кусочек текста. Наша задача - скачивать контент по этому адресу.

In [86]:
import requests

STEPS = 100

def download():
    requests.get("http://127.0.0.1:8000").text

# Простая программа, загружающая контент URL-странички. Типичная IO-bound задача.
def worker(steps):
    for i in range(steps):
        download()

%timeit -n1 worker(STEPS)

1 loop, best of 3: 11.1 s per loop


<div style="float:left;margin:0 10px 10px 0" markdown="1">![title](img/g1.png)</div>

In [93]:
import threading

NUM_THREADS = 64

def worker(steps):
    count = 0
    for i in range(steps):
        download()

def run_worker_threaded():    
    threads = []

    for i in range(NUM_THREADS):
        t = threading.Thread(target=worker, args=(STEPS//NUM_THREADS,))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

%timeit -n1 run_worker_threaded()

1 loop, best of 3: 452 ms per loop


<div style="float:left;margin:0 10px 10px 0" markdown="1">![title](img/g2.png)</div>

Ради интереса попробуем мультипроцессинг для этой задачи:

In [90]:
import multiprocessing

NUM_PROCESSES = 16

def worker(steps):
    count = 0
    for i in range(steps):
        download()

def run_worker_in_processes():    
    processes = []

    for i in range(NUM_PROCESSES):
        p = multiprocessing.Process(target=worker, args=(STEPS//NUM_PROCESSES,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

%timeit -n1 run_worker_in_processes()

1 loop, best of 3: 842 ms per loop


Как мы видим - треды позволили получить лучший результат (Macbook Air 2011 - 64 треда).

Чтобы упростить работу с тредами в Python есть модуль concurrent.futures: 
он предоставляет доступ к 2-м высокоуровневым объектам: ThreadPoolExecutor и ProcessPoolExecutor

In [192]:
import concurrent.futures
import requests

STEPS = 100

def download():
    return requests.get("http://127.0.0.1:8000").text

def run_in_executor():
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=64)

    future_to_url = {executor.submit(download): i for i in range(STEPS)}
    for future in concurrent.futures.as_completed(future_to_url):
        i = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%d generated an exception: %s' % (i, exc))
        else:
            print('%d page is %d bytes' % (i, len(data)))

    executor.shutdown()

run_in_executor()
#%timeit -n1 run_in_executor()

14 page is 383 bytes
18 page is 383 bytes
20 page is 383 bytes
9 page is 383 bytes
12 page is 382 bytes
21 page is 383 bytes
13 page is 383 bytes
30 page is 382 bytes
17 page is 383 bytes
8 page is 383 bytes
27 page is 383 bytes
24 page is 383 bytes
7 page is 383 bytes
6 page is 382 bytes
28 page is 381 bytes
5 page is 382 bytes
0 page is 382 bytes
4 page is 382 bytes
25 page is 383 bytes
38 page is 383 bytes
36 page is 382 bytes
26 page is 382 bytes
23 page is 383 bytes
40 page is 383 bytes
15 page is 381 bytes
2 page is 383 bytes
39 page is 383 bytes
32 page is 383 bytes
35 page is 383 bytes
16 page is 383 bytes
11 page is 383 bytes
22 page is 383 bytes
34 page is 382 bytes
31 page is 383 bytes
19 page is 383 bytes
37 page is 383 bytes
3 page is 381 bytes
1 page is 383 bytes
29 page is 381 bytes
33 page is 383 bytes
10 page is 383 bytes
43 page is 382 bytes
44 page is 383 bytes
47 page is 383 bytes
42 page is 382 bytes
49 page is 382 bytes
48 page is 383 bytes
51 page is 383 bytes
45

Аналогично можно использовать ProcessPoolExecutor, чтобы вынести работу в пул процессов. 

## Сложность многопоточных приложений

In [171]:
counter = 0

def worker(num):
    global counter
    for i in range(num):
        counter += 1

worker(1000000)

print(counter)

1000000


In [231]:
import threading

counter = 0

def worker(num):
    global counter
    for i in range(num):
        counter += 1

threads = []
for i in range(10):
    t = threading.Thread(target=worker, args=(100000,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(counter)

846875


In [167]:
import threading

counter = 0
lock = threading.Lock()

def worker(num):
    global counter
    for i in range(num):
        lock.acquire()
        counter += 1
        lock.release()

threads = []
for i in range(10):
    t = threading.Thread(target=worker, args=(100000,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(counter)

1000000


In [226]:
# deadlock example
import threading

counter = 0
lock = threading.Lock()

def print_counter():
    lock.acquire()
    print(counter)
    lock.release()

def worker():
    global counter
    lock.acquire()
    print_counter()
    counter += 1
    lock.release()
    
worker()

KeyboardInterrupt: 

**Вернемся к примеру с загрузкой URL: можем ли мы сделать еще лучше?**

* загружать странички еще быстрее
* потреблять меньше памяти, не расходуя ее на создание потоков
* не задумываться о синхронизации потоков

# Асинхронное программирование

Мотивация - IO-операции очень медленные, нужно заставить программу выполнять полезную работу во время ожидания ввода-вывода.

Сравнение Latency некоторых операций (https://gist.github.com/jboner/2841832)
<table align="left">
    <tr>
        <td>L1 CPU cache reference</td>
        <td>0.5 ns</td>
        <td></td>
    </tr>
    <tr>
        <td>Main memory reference</td>
        <td>100 ns</td>
        <td>200x L1 cache</td>
    </tr>
    <tr>
        <td>Read 1 MB sequentially from memory</td>
        <td>250,000 ns (250 us)</td>
        <td></td>
    </tr>
    <tr>
        <td>Round trip within same datacenter</td>
        <td>500,000 ns (500 us)</td>
        <td></td>
    </tr>
    <tr>
        <td>Read 1 MB sequentially from SSD*</td>
        <td>1,000,000 ns (1,000 us, 1ms)</td>
        <td></td>
    </tr>
    <tr>
        <td>Read 1 MB sequentially from disk</td>
        <td>20,000,000 ns (20,000 us, 20 ms)</td>
        <td>80x memory, 20X SSD</td>
    </tr>
    <tr>
        <td>Send packet CA->Netherlands->CA</td>
        <td>150,000,000 ns (150,000 us, 150 ms)</td>
        <td></td>
    </tr>    
</table>

In [15]:
import time

def cat():
    for i in range(3):
        print("Мяу")
        time.sleep(1)

def dog():
    for i in range(3):    
        print("Гав")
        time.sleep(1)
        
cat()
dog()

Мяу
Мяу
Мяу
Гав
Гав
Гав


Асинхронное программирование строится вокруг понятия Event Loop - "цикл событий". Event loop является основным координатором в асинхронных программах на Python. Он отвечает за:

* вызов нужных callback-функций в нужный момент
* регистрацию отложенных вызовов и таймеров

Это позволяет писать программы так, что в момент блокирующих IO операций контекст выполнения будет переключаться на другие задачи, ждущие выполнения.

В Python модуль asyncio предоставляет как раз такой Event Loop. Также в языке присутствуют специальные ключевые слова, позволяющие программировать в асинхронном стиле: **async** и **await**

**async** - ключевое слово, позволяющее обозначить функцию как асинхронную (корутина, coroutine). Такая функция может прервать свое выполнение в определенной точке (на блокирующей операции), а затем, дождавшись результата этой операции, продолжить свое выполнение.

**await** позволяет запустить такую функцию и дождаться результата.

**Чтобы программа стала работать асинхронно нужно использовать примитивы, которые есть в библиотеке asyncio**.

In [18]:
import asyncio
import time

async def cat():
    for i in range(3):
        print("Мяу")
        time.sleep(1)

async def dog():
    for i in range(3):
        print("Гав")
        time.sleep(1)

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(cat(), dog()))

Гав
Гав
Гав
Мяу
Мяу
Мяу


[None, None]

In [23]:
import asyncio

async def cat():
    for i in range(3):
        print("Мяу")
        await asyncio.sleep(1)

async def dog():
    for i in range(3):
        print("Гав")
        await asyncio.sleep(1)

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(cat(),dog()))

Мяу
Гав
Мяу
Гав
Мяу
Гав


[None, None]

Примеры других реализаций Event Loop'ов:

* Tornado IOLoop (https://github.com/tornadoweb/tornado)
* Twisted reactor (https://twistedmatrix.com/trac/)
* pyuv (https://github.com/saghul/pyuv)
* PyGame (http://pygame.org/hifi.html)
* ...

Мы еще вернемся к asyncio в лекции про интернет и клиент-серверные приложения. В том числе подробно посмотрим на сетевые операции - неблокирующее чтение и запись в сокеты.

Список библиотек, написанных поверх asyncio - https://github.com/python/asyncio/wiki/ThirdParty

In [15]:
import aiohttp
import asyncio

STEPS = 100

async def download():
    async with aiohttp.get("http://127.0.0.1:8000", loop=loop) as response:
        return await response.text()

async def worker(steps, loop):
    await asyncio.gather(*[download() for x in range(steps)])

loop = asyncio.get_event_loop()
%timeit -n1 loop.run_until_complete(worker(STEPS, loop))

1 loop, best of 3: 247 ms per loop


![title](img/g5.png)

# Завершающий пример (asyncio + multiprocessing)

In [29]:
# asyncio + multiprocessing
import aiohttp
import asyncio
import multiprocessing

NUM_PROCESSES = 2

STEPS = 100

async def download(loop):
    async with aiohttp.get("http://127.0.0.1:8000", loop=loop) as response:
        return await response.text()

async def worker(steps, loop):
    await asyncio.gather(*[download(loop) for x in range(steps)], loop=loop)

def run(steps):
    loop = asyncio.new_event_loop()
    loop.run_until_complete(worker(steps, loop))

def run_in_processes():    
    processes = []

    for i in range(NUM_PROCESSES):
        p = multiprocessing.Process(target=run, args=(STEPS//NUM_PROCESSES,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

%timeit -n1 run_in_processes()

1 loop, best of 3: 210 ms per loop
